## NeXtCortex project

The installation of the packages DrWatson, SpikingNeuralNetworks, UnPack, Logging, Plots, Statistics, Random, CSV, DataFrames, Unitful are required to run this file.

The file src/networks_utils.jl is also required.

In [1]:
using DrWatson
findproject(@__DIR__) |> quickactivate

include("src/network_utils.jl")
using .NetworkUtils

using SpikingNeuralNetworks
using UnPack
using Logging
using Plots
using Statistics
using Random
using CSV
using DataFrames
import SpikingNeuralNetworks: @update

global_logger(NullLogger())
SNN.@load_units

import SpikingNeuralNetworks: PoissonLayer, monitor!, sim!, SingleExpSynapse, IFParameter, PostSpike, STTC

img_path = "plots_and_images";

#### Network baseline configuration

In [2]:
TC3inhib_network = (
    Npop = (TE=200, CE=4000, PV=400, SST=300, VIP=300),
    seed = 1234,

    exc = IFParameter(
        τm=200pF / 10nS,
        El=-70mV,
        Vt=-50mV,
        Vr=-70mV,
        R=1/10nS,
    ),

    inh_PV  = IFParameter(τm=100pF/10nS, El=-70mV, Vt=-53mV, Vr=-70mV, R=1/10nS),
    inh_SST = IFParameter(τm=200pF/10nS, El=-70mV, Vt=-53mV, Vr=-70mV, R=1/10nS),
    inh_VIP = IFParameter(τm=150pF/10nS, El=-70mV, Vt=-53mV, Vr=-70mV, R=1/10nS),

    spike     = PostSpike(τabs=5ms),
    spike_PV  = PostSpike(τabs=2ms),
    spike_SST = PostSpike(τabs=10ms),
    spike_VIP = PostSpike(τabs=5ms),

    synapse     = SingleExpSynapse(τi=5ms, τe=5ms, E_i=-80mV, E_e=0mV),
    synapse_PV  = SingleExpSynapse(τi=5ms, τe=5ms, E_i=-80mV, E_e=0mV),
    synapse_SST = SingleExpSynapse(τi=12ms, τe=5ms, E_i=-80mV, E_e=0mV),
    synapse_VIP = SingleExpSynapse(τi=7ms, τe=5ms, E_i=-80mV, E_e=0mV),

    connections = (
        TE_to_CE = (p=0.05, μ=4nS, rule=:Fixed),
        TE_to_PV = (p=0.05, μ=4nS, rule=:Fixed),

        CE_to_CE = (p=0.05, μ=2nS, rule=:Fixed),
        CE_to_PV = (p=0.05, μ=2nS, rule=:Fixed),
        CE_to_TE = (p=0.05, μ=2nS, rule=:Fixed),
        CE_to_SST = (p=0.05, μ=2nS, rule=:Fixed),
        CE_to_VIP = (p=0.05, μ=2nS, rule=:Fixed),

        PV_to_CE  = (p=0.05, μ=10nS, rule=:Fixed),
        PV_to_PV  = (p=0.05, μ=10nS, rule=:Fixed),
        PV_to_SST = (p=0.05, μ=10nS, rule=:Fixed),

        SST_to_CE  = (p=0.025, μ=10nS, rule=:Fixed),
        SST_to_PV  = (p=0.025, μ=10nS, rule=:Fixed),
        SST_to_VIP = (p=0.025, μ=10nS, rule=:Fixed),

        VIP_to_SST = (p=0.3, μ=10nS, rule=:Fixed),
    ),

    afferents_to_TE  = (layer=PoissonLayer(rate=1.5Hz, N=1000), conn=(p=0.05, μ=4nS, rule=:Fixed)),
    afferents_to_CE  = (layer=PoissonLayer(rate=1.5Hz, N=1000), conn=(p=0.02, μ=4nS, rule=:Fixed)),
    afferents_to_PV  = (layer=PoissonLayer(rate=1.5Hz, N=1000), conn=(p=0.02, μ=4nS, rule=:Fixed)),
    afferents_to_SST = (layer=PoissonLayer(rate=1.5Hz, N=1000), conn=(p=0.15, μ=2nS, rule=:Fixed)),
    afferents_to_VIP = (layer=PoissonLayer(rate=1.5Hz, N=1000), conn=(p=0.10, μ=2nS, rule=:Fixed)),
);

In [3]:
# Build and simulate
model = NetworkUtils.build_network(TC3inhib_network)
monitor!(model.pop, [:v], sr=1kHz)


Random.seed!(TC3inhib_network.seed)
sim!(model, 3s);

"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"


In [ ]:
rplt, zrplt, frplt, vplt = NetworkUtils.analysis(model)

#Display of the baseline network activity
plts = plot(rplt, zrplt, frplt, vplt, layout=(2,2), size=(1000,1000))
display(plts)
savefig(plts, "$img_path/Baseline tight_layout.png")

#### Epileptic-like activity (increasing CE_to_CE)

In [5]:
TC3inhib_network_modified = (; TC3inhib_network..., 
    connections = (; TC3inhib_network.connections..., 
        CE_to_CE = (; TC3inhib_network.connections.CE_to_CE..., p = 0.15)
    )
)

model = NetworkUtils.build_network(TC3inhib_network_modified)
monitor!(model.pop, [:v], sr=1kHz)
Random.seed!(TC3inhib_network_modified.seed)
sim!(model, 3s);

"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"


In [ ]:
rplt, zrplt, frplt, vplt = NetworkUtils.analysis(model, name = "Epileptic-like state")

#Display of the network activity when connections.CE_to_CE.p is increased
plts = plot(rplt, zrplt, frplt, vplt, layout=(2,2), size=(1000,1000))
display(plts)
savefig(plts, "$img_path/Baseline tight_layout.png")

UndefVarError: UndefVarError: `analysis` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

#### Thalamic increased connection (increasing TE_to_CE)

In [ ]:
p_values = [0.10, 0.15]

for p in p_values
    TC3inhib_network_modified = (; TC3inhib_network..., 
        connections = (; TC3inhib_network.connections..., 
            TE_to_CE = (; TC3inhib_network.connections.TE_to_CE..., p = p)
        )
    )

    model = NetworkUtils.build_network(TC3inhib_network_modified)
    monitor!(model.pop, [:v], sr=1kHz)
    Random.seed!(TC3inhib_network_modified.seed)
    sim!(model, 3s)

    rplt, zrplt, frplt, vplt = NetworkUtils.analysis(model, name = "Thalamic increase p=$p");
end

"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"


UndefVarError: UndefVarError: `analysis` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [8]:
#Display of the network activity when connections.TE_to_CE.p is the last value of p_values 
plts = plot(rplt, zrplt, frplt, vplt, layout=(2,2), size=(1000,1000))
display(plts)
savefig(plts, "$img_path/Baseline tight_layout.png")

UndefVarError: UndefVarError: `rplt` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

#### Modulations Experiments

In [ ]:
pops_to_modify = (:VIP_to_SST, :PV_to_CE, :SST_to_CE, :TE_to_CE)

μ_values = [0.1, 0.5, 1.0, 1.5, 2.0, 5.0, 10]
p_values = [0.1, 0.2, 0.7, 1.0];

4-element Vector{Float64}:
 0.1
 0.2
 0.7
 1.0

In [21]:
for pop in pops_to_modify

    T = [ NetworkUtils.average_transition_time(TC3inhib_network, pop, μ, p)
      for μ in μ_values, p in p_values ]

    transition_heatmap = heatmap(
        p_values, μ_values, T,
        xlabel = "p scale",
        ylabel = "μ scale",
        title  = "Transition time to synchrony for pop $pop",
        color  = :viridis,
        clims  = (0, 3)
    )

    savefig(transition_heatmap,
            "$img_path/Heatmap modulation $pop transition_time.png")

end

"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSpikingSynapse"
"syn" * syn.name = "synSp

Excessive output truncated after 524298 bytes.

"syn" * syn.name = "synSpikingSynapse"

#### Slower inhibition test (increasing PV membrane time constant)

In [ ]:
TC3inhib_network_modified = (; TC3inhib_network..., 
    synapse_PV  = SingleExpSynapse(τi=20ms, τe=5ms, E_i=-80mV, E_e=0mV),
)

model = NetworkUtils.build_network(TC3inhib_network_modified)
monitor!(model.pop, [:v], sr=1kHz)
Random.seed!(TC3inhib_network_modified.seed)
sim!(model, 3s);

In [ ]:
rplt, zrplt, frplt, vplt = NetworkUtils.analysis(model, name = "Slower inhibition")

#Display of the network activity when connections.CE_to_CE.p is increased
plts = plot(rplt, zrplt, frplt, vplt, layout=(2,2), size=(1000,1000))
display(plts)
savefig(plts, "$img_path/Baseline tight_layout.png")